## Download GSV Pano and depth map for Launceston using streetlevel python library
* streetlevel is a library for downloading panoramas and metadata from street-level imagery services such as Google Street View, Apple Look Around, and several others. To install the package use: _pip install streetlevel_

* This notebook is basically same with the notebook to download GSV data for Wagga, but slightly different in a few parameters due to the difference in attribute fields in the ground-truth building points.

## Load modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from streetlevel import streetview
import geopandas as gpd
import json
import os
import re
from PIL import Image
from glob import glob
from aiohttp import ClientSession
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Set input and output folders
* A vector of building points
* Output folders for Panorama images and depth maps

In [3]:
add_points=r'C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\GA_data_documentation\Launceston Exposure Data\LC_Final\LC_Final.shp'
output_folder_pano=r'D:\Launceston\GSV\Pano'
output_folder_depth=r'D:\Launceston\GSV\Depth'
# output_folder_pano=r'C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\Panos\streetlevel'
# output_folder_depth=r'C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\Depth_decoded\streetlevel'

## Load GNAF points

In [4]:
gdf_points=gpd.read_file(add_points).to_crs('epsg:4326')
gdf_points.head()

,EASTING,NORTHING,UFI,ZONE,DEM,LID,ADDRESS,SUBURB,POSTCODE,FEATURE_NA,...,Min_OBJECT,LCC_FLOOR,check1,POINT_X_1,POINT_Y_1,ROWS,SA11,GVULN,GVULV,geometry
0,510962.988188,5.412541e+06,1002,15.0 Urban Mixed Use,2.697813,GNAF_GATAS702308914,13 PARK STREET,LAUNCESTON,7250,None,...,1061.0,3.12,0.0,147.131228,-41.438625,1.0,6.020110e+10,4.0,10.0,POINT (147.13123 -41.43862)
1,511021.688427,5.412248e+06,33,11.0 Inner Residential,4.208417,GNAF_GATAS702204375,293 BRISBANE STREET,LAUNCESTON,7250,None,...,0.0,3.33,0.0,147.131936,-41.441268,1.0,6.020110e+10,4.0,10.0,POINT (147.13194 -41.44127)
2,511082.136711,5.412226e+06,31,11.0 Inner Residential,4.540370,GNAF_GATAS702478623,4 MIDDLE STREET,LAUNCESTON,7250,None,...,0.0,3.70,0.0,147.132660,-41.441464,1.0,6.020110e+10,4.0,10.0,POINT (147.13266 -41.44146)
3,511068.760790,5.412219e+06,28,11.0 Inner Residential,5.212643,GNAF_GATAS702296648,6 MIDDLE STREET,LAUNCESTON,7250,None,...,0.0,3.37,0.0,147.132500,-41.441522,1.0,6.020110e+10,4.0,10.0,POINT (147.13250 -41.44152)
4,511048.726805,5.412229e+06,26,11.0 Inner Residential,5.162260,GNAF_GATAS702290011,8 MIDDLE STREET,LAUNCESTON,7250,None,...,0.0,3.37,0.0,147.132260,-41.441433,1.0,6.020110e+10,4.0,10.0,POINT (147.13226 -41.44143)


### View points

In [5]:
gdf_points.explore(column='LCC_FLOOR')

In [6]:
gdf_points['Floor_height']=gdf_points['LCC_FLOOR']

## Test workflow with first point
### Get point coordinates

In [7]:
i=0
lon=gdf_points.geometry.iloc[i].x
lat=gdf_points.geometry.iloc[i].y

### Record ground-truth FFH

In [8]:
ffh_gt=gdf_points.iloc[i]['Floor_height']
ffh_gt

3.12

### Search and download panorama without depth

In [9]:
zoom=5
pano = streetview.find_panorama(lat, lon)
pano

YNLZF16ubLTqrzW7SkVC3A (-41.43855, 147.13145) [2019-10]

In [10]:
pano.elevation

5.546634674072266

In [11]:
if pano is None:
    print('cannot find pano for location',lat,lon)
else:
    out_file_pano=os.path.join(output_folder_pano,pano.id+'.jpg')
    if os.path.exists(out_file_pano):
        print('pano file exists, skipping...')
    else:
        print('downloading to file ',out_file_pano)
        async with ClientSession() as session:
            await streetview.download_panorama_async(pano,out_file_pano,session,zoom=zoom)

downloading to file  D:\Launceston\GSV\Pano\YNLZF16ubLTqrzW7SkVC3A.jpg


### Download depth map using pano id

In [12]:
pano_with_depth = streetview.find_panorama_by_id(pano.id,download_depth=True)
depth_arr=pano_with_depth.depth.data
depth_img = Image.fromarray(depth_arr)
# plt.imshow(depth_arr)

In [13]:
depth_arr.shape

(256, 512)

Allow large image to be loaded:

In [14]:
Image.MAX_IMAGE_PIXELS=None

### Upscale depth map to the same dimension as pano (optional)

In [15]:
# if os.path.exists(out_file_pano):
#     with Image.open(out_file_pano) as img:
#         pano_size=img.size
# resampled_depth = depth_img.resize(pano_size, resample=Image.BILINEAR)

### Save depth map

In [16]:
out_file_depth=os.path.join(output_folder_depth,pano.id+'.tif')
if os.path.exists(out_file_depth):
    print('pano file exists, skipping...')
else:
    # resampled_depth.save(out_file, format='TIFF',compression='tiff_deflate')
    depth_img.save(out_file_depth, format='TIFF',compression='tiff_deflate')

## Batch querying pano metadata

In [17]:
gdf_points_meta=gdf_points.copy()
gdf_points_meta['pano_id']=pd.NA
gdf_points_meta['lat_c']=pd.NA
gdf_points_meta['lng_c']=pd.NA
gdf_points_meta['heading']=pd.NA
gdf_points_meta['pitch']=pd.NA
gdf_points_meta['roll']=pd.NA
gdf_points_meta['elevation']=pd.NA
gdf_points_meta['date']=pd.NA


In [18]:
for i in range(len(gdf_points)):
    lon=gdf_points.geometry.iloc[i].x
    lat=gdf_points.geometry.iloc[i].y
    pano = streetview.find_panorama(lat, lon)
    if not pano is None:
        gdf_points_meta.at[i,'pano_id']=pano.id
        gdf_points_meta.at[i,'lat_c']=pano.lat
        gdf_points_meta.at[i,'lng_c']=pano.lon
        gdf_points_meta.at[i,'heading']=pano.heading
        gdf_points_meta.at[i,'pitch']=pano.pitch
        gdf_points_meta.at[i,'roll']=pano.roll
        gdf_points_meta.at[i,'elevation']=pano.elevation
        gdf_points_meta.at[i,'date']=pano.date
    else:
        print('cannot find pano for location',lat,lon)
gdf_points_meta

cannot find pano for location -41.42497799999999 147.12215799999998
cannot find pano for location -41.426159874099994 147.12372464999999
cannot find pano for location -41.424454999999995 147.12429099999997
cannot find pano for location -41.424494999999986 147.12318399999998
cannot find pano for location -41.42407099999999 147.12577799999997
cannot find pano for location -41.429205 147.125652
cannot find pano for location -41.42757799999999 147.12440799999996
cannot find pano for location -41.41658899999999 147.13120299999997
cannot find pano for location -41.417378 147.13116099999996
cannot find pano for location -41.41726299999998 147.14194699999996
cannot find pano for location -41.41844499999999 147.14206599999997
cannot find pano for location -41.410308999999984 147.13431899999998
cannot find pano for location -41.429657999999996 147.14524299999997
cannot find pano for location -41.42748299999999 147.15009899999998
cannot find pano for location -41.42756899999999 147.15038699999997

,EASTING,NORTHING,UFI,ZONE,DEM,LID,ADDRESS,SUBURB,POSTCODE,FEATURE_NA,...,geometry,Floor_height,pano_id,lat_c,lng_c,heading,pitch,roll,elevation,date
0,510962.988188,5.412541e+06,1002,15.0 Urban Mixed Use,2.697813,GNAF_GATAS702308914,13 PARK STREET,LAUNCESTON,7250,None,...,POINT (147.13123 -41.43862),3.12,YNLZF16ubLTqrzW7SkVC3A,-41.438551,147.131445,5.841477,0.022252,6.228637,5.546635,2019-10
1,511021.688427,5.412248e+06,33,11.0 Inner Residential,4.208417,GNAF_GATAS702204375,293 BRISBANE STREET,LAUNCESTON,7250,None,...,POINT (147.13194 -41.44127),3.33,Tz6NjkiBNSG1ge0lKjXhdg,-41.441052,147.131782,1.040822,0.037883,6.235322,5.922997,2023-09
2,511082.136711,5.412226e+06,31,11.0 Inner Residential,4.540370,GNAF_GATAS702478623,4 MIDDLE STREET,LAUNCESTON,7250,None,...,POINT (147.13266 -41.44146),3.70,hANXkavpLZzicH-g505L6g,-41.441531,147.132669,0.957574,0.039241,0.022607,7.15661,2023-09
3,511068.760790,5.412219e+06,28,11.0 Inner Residential,5.212643,GNAF_GATAS702296648,6 MIDDLE STREET,LAUNCESTON,7250,None,...,POINT (147.13250 -41.44152),3.37,kC4sWh0ZUf1E7nz52Fb9xA,-41.441585,147.132567,0.963067,0.051902,0.010492,7.739836,2023-09
4,511048.726805,5.412229e+06,26,11.0 Inner Residential,5.162260,GNAF_GATAS702290011,8 MIDDLE STREET,LAUNCESTON,7250,None,...,POINT (147.13226 -41.44143),3.37,hw9Rgp7d5H3mb-WlVW-Z_A,-41.441662,147.132419,0.959565,0.034847,0.021449,8.384348,2023-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,511678.913227,5.415060e+06,1415,23.0 Commercial,2.530000,GNAF_GATAS716623620,26 OSWALD STREET,INVERMAY,7248,None,...,POINT (147.13975 -41.41592),2.61,zlExcFoRXIFgqEMdWbk8Jw,-41.416136,147.139393,2.542899,-0.008668,0.015574,6.60992,2023-09
1272,511662.145037,5.415079e+06,1416,23.0 Commercial,2.383714,GNAF_GATAS702465036,28 OSWALD STREET,INVERMAY,7248,None,...,POINT (147.13955 -41.41576),2.61,q57A-gkokKQu757cn3CV0A,-41.415947,147.139221,2.54142,-0.025394,6.271691,6.408826,2023-09
1273,511635.268231,5.415099e+06,1417,23.0 Commercial,2.317272,GNAF_GATAS702317895,32 OSWALD STREET,INVERMAY,7248,None,...,POINT (147.13923 -41.41557),2.48,TgdWgi_3bubHkVsiWDj7RQ,-41.415716,147.139029,2.563032,-0.02469,6.264193,5.787523,2023-09
1274,511690.687378,5.415003e+06,870,23.0 Commercial,3.437857,GNAF_GATAS702219883,40 BURNS STREET,INVERMAY,7248,None,...,POINT (147.13989 -41.41644),4.12,CsBIbz8-ybkm1cCmVFIIVA,-41.416494,147.139735,2.471215,-0.021353,6.247352,7.106176,2023-09


## Save the metadata

In [19]:
gdf_points_meta['date'] = gdf_points_meta['date'].astype('string')
gdf_points_meta['heading'] = gdf_points_meta['heading'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['lat_c'] = gdf_points_meta['lat_c'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['lng_c'] = gdf_points_meta['lng_c'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['pitch'] = gdf_points_meta['pitch'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['roll'] = gdf_points_meta['roll'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['elevation'] = gdf_points_meta['elevation'].replace(pd.NA, np.nan).astype('float')

C:\Users\lliu\AppData\Local\Temp\ipykernel_16320\3730501706.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf_points_meta['heading'] = gdf_points_meta['heading'].replace(pd.NA, np.nan).astype('float')
C:\Users\lliu\AppData\Local\Temp\ipykernel_16320\3730501706.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf_points_meta['lat_c'] = gdf_points_meta['lat_c'].replace(pd.NA, np.nan).astype('float')
C:\Users\lliu\AppData\Local\Temp\ipykernel_16320\3730501706.py:4: FutureWarning: Downcasting behavior in `replace` is deprecat

In [20]:
outfile_meta='D:\Launceston\GSV\LC_Final_meta.geojson'
gdf_points_meta.to_file(outfile_meta, driver="GeoJSON")
# to avoid overwriting
# if not os.path.exists(outfile_meta):
#     gdf_points_meta.to_file(outfile_meta, driver="GeoJSON")

## Batch downloading

In [21]:
pano_ids=gdf_points_meta['pano_id'][gdf_points_meta['pano_id'].notna()]
len(pano_ids)

1251

In [22]:
for pano_id in pano_ids:
    out_file_pano=os.path.join(output_folder_pano,pano_id+'.jpg')
    if os.path.exists(out_file_pano):
        print('pano file exists, skipping...')
    else:
        pano=streetview.find_panorama_by_id(pano_id,download_depth=False)
        if pano is None:
            print('cannot find pano')
        else:
            try:
                print('downloading to file ',out_file_pano)
                async with ClientSession() as session:
                    await streetview.download_panorama_async(pano,out_file_pano,session,zoom=zoom)
            except Exception as e:
                print(e)

    out_file_depth=os.path.join(output_folder_depth,pano_id+'.tif')
    if os.path.exists(out_file_depth):
        print('depth map exists, skipping...')
    else:
        try:
            pano_with_depth = streetview.find_panorama_by_id(pano_id,download_depth=True)
            if pano_with_depth is None:
                print('cannot decode depth map pano ',pano_id)
            else:
                print('decoding depth map')
                depth_arr=pano_with_depth.depth.data
                depth_img = Image.fromarray(depth_arr)
                # resampled_depth.save(out_file, format='TIFF',compression='tiff_deflate')
                depth_img.save(out_file_depth, format='TIFF',compression='tiff_deflate')
        except Exception as e:
            print(e)

pano file exists, skipping...
depth map exists, skipping...
downloading to file  D:\Launceston\GSV\Pano\Tz6NjkiBNSG1ge0lKjXhdg.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\hANXkavpLZzicH-g505L6g.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\kC4sWh0ZUf1E7nz52Fb9xA.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\hw9Rgp7d5H3mb-WlVW-Z_A.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\OH5vAtnkf1rJ0PBWned9Gw.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\GvP_WNzTHiz53PaBhohUcw.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\yUC9VsjcbhJLP9IhkcS9NA.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\1dWomAWXYA5WBVKOUSsupg.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\04NHb7NoIVI3bCJWr7dqxA.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\Pano\ofzSLVU6ziUOhvOKVoIdlA.jpg
decoding depth map
downloading to file  D:\Launceston\GSV\P